In [8]:
#### Elliptic Curve Cryptography
p_list = [(192, 105), (17, 56), (200, 119), (1, 193), (42, 99)]
for p in p_list:
    print((p[1] ** 2) % 223)
    print((p[0] ** 3 + 7) % 223)

98
98
14
14
112
105
8
8
212
59


In [1]:
from ecc import FieldElement, Point
a = FieldElement(num = 0, prime = 223)
b = FieldElement(7, 223)
x = FieldElement(192, 223)
y = FieldElement(105, 223)
p1 = Point(x, y, a, b)
print(p1)

Point(192,105)_0_7 FieldElement(223)


In [2]:
from unittest import TestCase

class ECCTest(TestCase):
    def test_on_curve(self):
        prime = 223
        a = FieldElement(0, prime)
        b = FieldElement(7, prime)
        valid_points = ((192 ,105), (17, 56), (1, 193))
        invalid_points = ((200, 119), (42, 99))
        for x_raw, y_raw in valid_points:
            x = FieldElement(x_raw, prime)
            y = FieldElement(y_raw, prime)
            Point(x, y, a, b)
        for x_raw, y_raw in invalid_points:
            x = FieldElement(x_raw, prime)
            y = FieldElement(y_raw, prime)
            with self.assertRaises(ValueError):
                Point(x, y, a, b)

In [4]:
from helper import run
run(ECCTest('test_on_curve'))

.
----------------------------------------------------------------------
Ran 1 test in 0.001s

OK


In [4]:
from ecc import FieldElement, Point
prime = 223
a = FieldElement(num = 0, prime = prime)
b = FieldElement(7, prime)
x1 = FieldElement(192, prime)
y1 = FieldElement(105, prime)
x2 = FieldElement(17, prime)
y2 = FieldElement(56, prime)
p1 = Point(x1, y1, a, b)
p2 = Point(x2, y2, a, b)
print(p1 + p2)

Point(170,142)_0_7 FieldElement(223)


In [4]:
x1 = FieldElement(170, prime)
y1 = FieldElement(142, prime)
x2 = FieldElement(60, prime)
y2 = FieldElement(139, prime)
p1 = Point(x1, y1, a, b)
p2 = Point(x2, y2, a, b)
print(p1 + p2)

Point(220,181)_0_7 FieldElement(223)


In [2]:
# 점 덧셈은 비선형이고 계산하기 쉽지 않음
# 스칼라 곱셈은 어렵지 않지만 나눗셈은 그렇지 않다
# 이를 이산로그문제(discrete log problem)이라 하고, 타원곡선 암호 방법의 원리임

# 스칼라 곱셈의 다른 성질은 어떤 점에 스칼라 값을 계속 증가시키면서 곱하면 무한원점(point at infinity)에 도달한다는 것
from ecc import FieldElement, Point
prime = 223
a = FieldElement(num = 0, prime = prime)
b = FieldElement(7, prime)
x = FieldElement(47, prime)
y = FieldElement(71, prime)
p = Point(x, y, a, b)
for s in range(1, 21):
    result = s * p
    print(f'{s} * (47, 71) = ({result.x.num}, {result.y.num})')

1 * (47, 71) = (47, 71)
2 * (47, 71) = (36, 111)
3 * (47, 71) = (15, 137)
4 * (47, 71) = (194, 51)
5 * (47, 71) = (126, 96)
6 * (47, 71) = (139, 137)
7 * (47, 71) = (92, 47)
8 * (47, 71) = (116, 55)
9 * (47, 71) = (69, 86)
10 * (47, 71) = (154, 150)
11 * (47, 71) = (154, 73)
12 * (47, 71) = (69, 137)
13 * (47, 71) = (116, 168)
14 * (47, 71) = (92, 176)
15 * (47, 71) = (139, 86)
16 * (47, 71) = (126, 127)
17 * (47, 71) = (194, 172)
18 * (47, 71) = (15, 86)
19 * (47, 71) = (36, 112)
20 * (47, 71) = (47, 152)


In [6]:
# 유한순환군 (finite cyclic group)
# 유한체에서 정의된 타원곡선 위 한 점에 스칼라 값을 곱해서 생성, 이 한 점을 생성점(generator point)라고 함

prime = 223
a = FieldElement(0, prime)
b = FieldElement(7, prime)
x = FieldElement(15, prime)
y = FieldElement(86 ,prime)
p = Point(x, y, a, b)
inf = Point(None, None, a, b)
product = p
count = 1
while product != inf:
    product += p
    count += 1
    print(product, count)

Point(139,86)_0_7 FieldElement(223) 2
Point(69,137)_0_7 FieldElement(223) 3
Point(69,86)_0_7 FieldElement(223) 4
Point(139,137)_0_7 FieldElement(223) 5
Point(15,137)_0_7 FieldElement(223) 6
Point(infinity) 7


In [8]:
gx = 0x79be667ef9dcbbac55a06295ce870b07029bfcdb2dce28d959f2815b16f81798
gy = 0x483ada7726a3c4655da4fbfc0e1108a8fd17b448a68554199c47d08ffb10d4b8
# p는 유한체의 위수
p = 2 ** 256 - 2 ** 32 - 977
# n은 군의 위수
n = 0xfffffffffffffffffffffffffffffffebaaedce6af48a03bbfd25e8cd0364141
x = FieldElement(gx, p)
y = FieldElement(gy, p)
seven = FieldElement(7, p)
zero = FieldElement(0, p)
G = Point(x, y, zero, seven)
print(n * G)

Point(infinity)


In [10]:
P = 2 ** 256 - 2 ** 32 - 977

class S256Field(FieldElement):

    def __init__(self, num, prime=None):
        super().__init__(num = num, prime = P)

    def __repr__(self):
        return f'{(self.num).zfill(64):x}'

In [11]:
A = 0
B = 7
N = 0xfffffffffffffffffffffffffffffffebaaedce6af48a03bbfd25e8cd0364141


class S256Point(Point):

    def __init__(self, x, y, a=None, b=None):
        a, b = S256Field(A), S256Field(B)
        if type(x) == int:
            super().__init__(x = S256Field(x), y = S256Field(y), a = a, b = b)
        else:
            super().__init__(x = x, y = y, a = a, b = b)

    def __rmul__(self, coefficient):
        coef = coefficient % N
        return super().__rmul__(coef)

G = S256Point(
    0x79be667ef9dcbbac55a06295ce870b07029bfcdb2dce28d959f2815b16f81798,
    0x483ada7726a3c4655da4fbfc0e1108a8fd17b448a68554199c47d08ffb10d4b8
)

In [12]:
print(N*G)

Point(infinity)


In [6]:
# ECDSA(Elliptic Curve Digital Signature Algorithm): 타원곡선 디지털 서명 알고리즘
from ecc import S256Point, G, N
z = 0xbc62d4b80d9e36da29c16c5d4d9f11731f36052c72401a76c23c0fb5a9b74423
r = 0x37206a0610995c58074999cb9767b87af4c4978db68c06e8e6e81d282047a7c6
s = 0x8ca63759c1157ebeaec0d03cecca119fc9a75bf8e6d0fa65c841c8e2738cdaec
px = 0x04519fac3d910ca7e7138f7013706f619fa8f033e6ec6e09370ea38cee6a7574
py = 0x82b51eab8c27c66e26c858a079bcdf4f1ada34cec420cafc7eac1a42216fb6c4
point = S256Point(px, py)
s_inv = pow(s, N-2, N)
u = z * s_inv % N
v = r * s_inv % N
print((u * G + v * point).x.num == r)

# u = z / s, v = r / s
# uG + vP = (r,y).x 즉 R의 x좌표와 동일한지 확인하여 서명 검증

True


In [12]:
from ecc import S256Point, G, N
point = S256Point(
    0x887387e452b8eacc4acfde10d9aaf7f6d9a0f975aabb10d006e4da568744d06c,
    0x61de6d95231cd89026e286df3b6ae4a894a3378e393e93a0f45b666329a0ae34)
z = 0xec208baa0fc1c19f708a9ca96fdeff3ac3f230bb4a7ba4aede4942ad003c0f60
r = 0xac8d1c87e51d0d441be8b3dd5b05c8795b48875dffe00b7ffcfac23010d3a395
s = 0x68342ceff8935ededd102dd876ffd6ba72d6a427a3edb13d26eb0781cb423c4
u = z * pow(s, N - 2, N) % N
v = r * pow(s, N - 2, N) % N
print((u * G + v * point).x.num == r)

True


In [13]:
class Signature:

    def __init__(self, r, s):
        self.r = r
        self.s = s
    
    def __repr(self):
        return f'Signature({self.r:x}, {self.s:x})'

In [1]:
from helper import hash256
from ecc import S256Point, G, N
e = int.from_bytes(hash256(b'my secret'), 'big')
z = int.from_bytes(hash256(b'my message'), 'big')
k = 1234567890
r = (k * G).x.num
k_inv = pow(k, N-2, N)
s = (z + r * e) * k_inv % N
point = e * G
print(point)
print(hex(z))
print(hex(r))
print(hex(s))

S256Point(028d003eab2e428d11983f3e97c3fa0addf3b42740df0d211795ffb3be2f6c52, 0ae987b9ec6ea159c78cb2a937ed89096fb218d9e7594f02b547526d8cd309e2)
0x231c6f3d980a6b0fb7152f85cee7eb52bf92433d9919b9c5218cb08e79cce78
0x2b698a0f0a4041b77e63488ad48c23e8e8838dd1fb7520408b121697b782ef22
0xbb14e602ef9e3f872e25fad328466b34e6734b7a0fcd58b1eb635447ffae8cb9


In [4]:
e = 12345
z = int.from_bytes(hash256(b'Programming Bitcoin!'), 'big')
k = 1234567890
r = (k * G).x.num
k_inv = pow(k, N - 2 , N)
s = (z + r * e) * k_inv % N
point = e * G
print(point)
print(hex(z))
print(hex(r))
print(hex(s))

S256Point(f01d6b9018ab421dd410404cb869072065522bf85734008f105cf385a023a80f, 0eba29d0f0c5408ed681984dc525982abefccd9f7ff01dd26da4999cf3f6a295)
0x969f6056aa26f7d2795fd013fe88868d09c9f6aed96965016e1936ae47060d48
0x2b698a0f0a4041b77e63488ad48c23e8e8838dd1fb7520408b121697b782ef22
0x1dbc63bfef4416705e602a7b564161167076d8b20990a0f26f316cff2cb0bc1a


In [5]:
from random import randint
class PrivateKey:

    def __init__(self, secret):
        self.secret = secret
        self.point = secret * G

    def hex(self):
        return f'{self.secret:x}'.zfill(64)

    def sign(self, z):
        k = randint(0, N)
        r = (k * G).x.num
        k_inv = pow(k, N - 2, N)
        s = (z + r * self.secret) * k_inv % N
        if s > N/2:
            s = N - s
        return Signature(r, s)
